# UI for RT1_Assignment3

In [1]:
# import
from IPython.display import display
from move_base_msgs.msg import MoveBaseActionGoal, MoveBaseGoal, MoveBaseActionFeedback
from geometry_msgs.msg import Twist
import actionlib
from actionlib_msgs.msg import GoalID
import rospy
import ipywidgets as widgets
from ipywidgets import Button, Layout, ButtonStyle, GridBox, VBox, HBox, interact, interactive, fixed, interact_manual
from __future__ import print_function
from jupyros import ros3d 
import jupyros as jr
import os
import matplotlib.pyplot as plt
import tf
import math
from nav_msgs.msg import Odometry
from rosgraph_msgs.msg import Clock
from sensor_msgs.msg import LaserScan
from nav_msgs.msg import OccupancyGrid
from tf.transformations import quaternion_matrix 
import numpy as np
from matplotlib.animation import FuncAnimation

In [2]:
rospy.init_node('UI_node_jupyter')

In [3]:
# definition for the variable array dimension
SIZE = 144

# definition of a threshold for the distance from obstacle
wth = 1

# Variable for the goal and the cancellation
g = MoveBaseActionGoal()
v = Twist()

# Variables for the linear and angular velocity
vLin = 0.5
vAng = 1.0
vMaxLin = 2.0
vMaxAng = 4.0
vMinLin = 0.25
vMinAng = 0.5

# Variable to store the number of times a goal is reached or not reached/cancelled
tot_reached = 0
tot_notReached = 0

# Variable to store the number of pression of a button
publish_bt = 0
cancel_bt = 0
enable_bt = 0
front_bt = 0
frontR_bt = 0
frontL_bt = 0
turnR_bt = 0
turnL_bt = 0
stop_bt = 0
back_bt = 0
incV_bt = 0
decV_bt = 0

# Bool
G = False

# Variable for the current goal position
xG = 0
yG = 0

# Threshold for the distance from the goal
th = 0.5

# variable defining the angle accuracy of the robot laser scan
angle_interval = 0.25

# Variable for the keyboard use (initialized to False)
# Goal publication: mod = False
# Keyboard use: mod = True
# Goal Cancellation: mod = False
mod = False;

In [4]:
# Functions for the pub and sub
def currGoal(m = MoveBaseActionGoal):
    global xG 
    global yG
    
    xG = m.goal.target_pose.pose.position.x;
    yG = m.goal.target_pose.pose.position.x;
    
def takeStatus(msg = MoveBaseActionFeedback):
    global goal_id
    global xG
    global yG
    global th
    global tot_reached
    
    goal_id = msg.status.goal_id.id;
    
    if(G):
        if(abs(msg.feedback.base_position.pose.position.x - xG) <= th and abs(msg.feedback.base_position.pose.position.y - yG) <= th):
            cancelGoal(True)
            tot_reached += 1
        
def cancelGoal(reached):
    global goal_id
    global G
    global tot_notReached
    
    if(G):
        goalToCancel = GoalID()
        goalToCancel.id = goal_id
        pubC.publish(goalToCancel)
        G = False
        if(not(reached)): # goal not reached
            tot_notReached += 1
            

In [5]:
# Publishers definition
pub = rospy.Publisher("/move_base/goal", MoveBaseActionGoal, queue_size = 1)
pubC = rospy.Publisher("/move_base/cancel", GoalID, queue_size = 1)
pubV = rospy.Publisher("/cmd_vel", Twist, queue_size = 1)

# Subscribers definition
subM = rospy.Subscriber("/move_base/feedback", MoveBaseActionFeedback, takeStatus)
subG = rospy.Subscriber("/move_base/goal", MoveBaseActionGoal, currGoal)
#subC = rospy.Subscriber("/map",LaserScan, driveAssist)

## Menù

#### UI widgets

In [6]:
# definition of the widgets for the position publication
xBox = widgets.BoundedFloatText(
    value = 0.0,
    min = -20.00,
    max = 20.0,
    step = 1,
    description = 'X:',
    disabled = False
)
yBox = widgets.BoundedFloatText(
    value = 0.0,
    min = -20.00,
    max = 20.0,
    step = 1,
    description = 'Y:',
    disabled = False
)

# definition of the buttons to enable/disable the avoid collision
en_bt = widgets.Button(
    description = 'ENABLE CONTROL',
    disabled = False,
    button_style='',
    tootltip = 'Click me',
    icon = '',
    style = ButtonStyle(font_weight='bold')
)

dis_bt = widgets.Button(
    description = 'DISABLE CONTROL',
    disabled = False,
    button_style='',
    tootltip = 'Click me',
    icon = '',
    style = ButtonStyle(font_weight='bold')
)

collision_row = widgets.HBox([en_bt, dis_bt])

# definition of the buttons
bc = widgets.Button(
    description = 'Cancel the goal',
    disabled = False,
    button_style='danger',
    tootltip = 'Click me',
    icon = ''
)
bt = widgets.Button(
    description = 'PUBLISH',
    disabled = False,
    button_style='',
    tootltip = 'Click me',
    icon = '',
    style = ButtonStyle(button_color = 'lightgreen',
                        font_weight='bold')
)


# Buttons for the keyboard drive
b_front = widgets.Button(
    description = 'FRONT',
    disabled = False,
    button_style='',
    tootltip = 'Click me',
    icon = '',
    style = ButtonStyle(button_color = 'lightblue',
                        font_weight='bold')
)
b_front_r = widgets.Button(
    description = 'FRONT-R',
    disabled = False,
    button_style='',
    tootltip = 'Click me',
    icon = '',
    style = ButtonStyle(button_color = 'lightblue',
                        font_weight='bold')
)
b_front_l = widgets.Button(
    description = 'FRONT-L',
    disabled = False,
    button_style='',
    tootltip = 'Click me',
    icon = '',
    style = ButtonStyle(button_color = 'lightblue',
                        font_weight='bold')
)
b_stop = widgets.Button(
    description = 'STOP',
    disabled = False,
    button_style='',
    tootltip = 'Click me',
    icon = '',
    style = ButtonStyle(button_color = 'red',
                        font_weight='bold')
)
b_turn_l = widgets.Button(
    description = 'TURN-L',
    disabled = False,
    button_style='',
    tootltip = 'Click me',
    icon = '',
    style = ButtonStyle(button_color = 'lightblue',
                        font_weight='bold')
)
b_turn_r = widgets.Button(
    description = 'TURN-R',
    disabled = False,
    button_style = '',
    tootltip = 'Click me',
    icon = '',
    style = ButtonStyle(button_color = 'lightblue',
                        font_weight='bold')
)
b_decrease = widgets.Button(
    description = 'DEC VEL',
    disabled = False,
    tootltip = 'Click me',
    icon = '',
    style = ButtonStyle(button_color = 'yellow',
                        font_weight='bold')
)
b_increase = widgets.Button(
    description = 'INC VEL',
    disabled = False,
    button_style = '',
    tootltip = 'Click me',
    icon = '',
    style = ButtonStyle(button_color = 'green',
                        font_weight='bold')
)
b_back = widgets.Button(
    description = 'BACK',
    disabled = False,
    button_style='',
    tootltip = 'Click me',
    icon = '',
    style = ButtonStyle(button_color = 'lightblue',
                        font_weight='bold')
)


# Buttons for the enable
en_keyboard = widgets.Button(
    description = 'ENABLE MANUAL',
    disabled = False,
    button_style='warning',
    tootltip = 'Click me',
    icon = '',
    style = ButtonStyle(font_weight='bold')
)

# Definition of the rows to show
first_row = widgets.HBox([b_front_l, b_front, b_front_r])
second_row = widgets.HBox([b_turn_l, b_stop, b_turn_r])
third_row = widgets.HBox([b_back, b_increase, b_decrease])

#### Widgets' function

In [7]:
# Functions for the click button of the keyboard
def front_vel(b_front):
    global pubV
    global vLin
    global v
    global mod
    global front_bt
    
    front_bt += 1
    
    if(mod):
        v.linear.x = vLin
        v.angular.z = 0.0
        pubV.publish(v)
    
def front_r_vel(b_front_r):
    global pubV
    global vLin
    global vAng
    global v
    global mod
    global frontR_bt
    
    frontR_bt += 1
    
    if(mod):
        v.linear.x = vLin
        v.angular.z = -vAng
        pubV.publish(v)
    
def front_l_vel(b_front_l):
    global pubV
    global vLin
    global vAng
    global v
    global mod
    global frontL_bt
    
    frontL_bt += 1
    
    if(mod):
        v.linear.x = vLin
        v.angular.z = vAng
        pubV.publish(v)
    
def turn_r_vel(b_turn_r):
    global pubV
    global vAng
    global v
    global mod
    global turnR_bt
    
    turnR_bt += 1
    
    if(mod):
        v.linear.x = 0.0
        v.angular.z = -vAng
        pubV.publish(v)
    
def turn_l_vel(b_turn_l):
    global pubV
    global vAng
    global v
    global mod
    global turnL_bt
    
    turnL_bt += 1
    
    if(mod):
        v.linear.x = 0.0
        v.angular.z = vAng
        pubV.publish(v)
    
def back_vel(b_back):
    global pubV
    global vLin
    global v
    global mod
    global back_bt
    
    back_bt += 1
    
    if(mod):
        v.linear.x = -vLin
        v.angular.z = 0.0
        pubV.publish(v)
    
def stop_vel(b_stop):
    global pubV
    global v
    global mod
    global stop_bt
    
    stop_bt += 1
    
    if(mod):
        v.linear.x = 0.0
        v.angular.z = 0.0
        pubV.publish(v)
    
def inc_vel(b_increase):
    global vLin
    global vAng
    global vMaxLin
    global vMaxAng
    global vMinLin
    global vMinAng
    global mod
    global incV_bt
    
    incV_bt += 1
    
    if(mod):
        if((vLin <= vMaxLin) and (vAng <= vMaxAng)):
            vLin += 0.1*vLin
            vAng += 0.1*vAng
        else:
            vLin = vMaxLin
            vAng = vMaxAng
    
def dec_vel(b_decrease):
    global vLin
    global vAng
    global vMaxLin
    global vMaxAng
    global vMinLin
    global vMinAng
    global mod
    global decV_bt
    
    decV_bt += 1
    
    if(mod):
        if((vLin >= vMinLin) and (vAng >= vMinAng)):
            vLin -= 0.1*vLin
            vAng -= 0.1*vAng
        else:
            vLin = vMinLin
            vAng = vMinAng
    
# Definition of the function for the button click
# Function for the button bt to publish the position
def pub_pos(bt):
    global G
    global pub
    global g
    global mod
    global v
    global pub
    global publish_bt
    
    publish_bt += 1
    
    if(mod):
        mod = False
    
    # if there is already a goal it will be cancelled
    if(G):
        cancelGoal(False)
    
    g.goal.target_pose.header.frame_id = "map"
    g.goal.target_pose.pose.position.x = xBox.value
    g.goal.target_pose.pose.position.y = yBox.value
    g.goal.target_pose.pose.orientation.w = 1.0
    pub.publish(g)
    G = True
    
# Function for the button bc to cancel the goal
def bc_observer(bc):
    global mod
    global cancel_bt
    
    cancel_bt += 1
    
    mod = False
    cancelGoal(False)

# Function for the button en_keybord
def key_fun(en_keyboard):
    global mod
    global enable_bt
    
    enable_bt += 1
    
    mod = True
    cancelGoal(False)

## Publish a (x,y) position

In [8]:
# Code fot the publication with the bt button
display(widgets.VBox([xBox, yBox]))
display(bt)

bt.on_click(pub_pos)

Button(description='PUBLISH', style=ButtonStyle(button_color='lightgreen', font_weight='bold'))

## Cancel the current goal

In [9]:
# Code to cancel the current goal set
display(bc)

bc.on_click(bc_observer)

Button(button_style='danger', description='Cancel the goal', style=ButtonStyle())

## Drive by keyboard

### In order to use the keyboard below, you first need to enable it by clicking the button.
#### To increment or decrement the velocity click the button and then press a command to make the differences visible

In [10]:
# Button to enable the keyboard use
display(en_keyboard)

en_keyboard.on_click(key_fun)

# Code to drive the robot by the keyboard use
display(widgets.VBox([first_row, second_row, third_row]))

b_front.on_click(front_vel)
b_front_r.on_click(front_r_vel)
b_front_l.on_click(front_l_vel)
b_turn_r.on_click(turn_r_vel)
b_turn_l.on_click(turn_l_vel)
b_back.on_click(back_vel)
b_stop.on_click(stop_vel)
b_increase.on_click(inc_vel)
b_decrease.on_click(dec_vel)

Button(button_style='warning', description='ENABLE MANUAL', style=ButtonStyle(font_weight='bold'))

## Map

In [11]:
# Definition of the elements needed for the map
view = ros3d.Viewer()
rc = ros3d.ROSConnection(url = "ws://localhost:9090") 
tf_client = ros3d.TFClient(ros = rc, fixed_frame = 'map')
laser_view = ros3d.LaserScan(topic = "/scan", ros = rc, tf_client = tf_client) 
map_view = ros3d.OccupancyGrid(topic = "/map", ros = rc, tf_client = tf_client) 
path = ros3d.Path(topic = "/move_base/NavfnROS/plan", ros = rc, tf_client = tf_client)
urdf = ros3d.URDFModel(ros = rc, tf_client = tf_client, path = os.environ.get('JUPYROS_ASSETS_URL', 'http://localhost:3000')) 
grid = ros3d.GridModel()
view.objects = [grid, laser_view, map_view, path, urdf]

# Map visualization
view

Viewer(objects=[GridModel(), LaserScan(ros=ROSConnection(url='ws://localhost:9090'), tf_client=TFClient(fixed_…

## Pie chart to store the amount of goal reached and not reached or canceled

In [12]:
%matplotlib widget

class PieVisualiser:
    def __init__(self):
        self.fig1, self.ax = plt.subplots()
        
    def pie_init(self):
        return self.ln
    
    def pie_callback(self, msg):
        global tot_reached
        global tot_notReached
        
        sizes = [tot_reached, tot_notReached]
        colors = ['green', 'red']
        patches, texts = self.ax.pie(sizes, colors = colors, startangle = 90)
        labels = ['Reacehd: (' + str(tot_reached) + ')' ,'Not Reacehd or Cancelled: (' + str(tot_notReached) + ')']
        
        explode = (0, 0)
        
        self.ax.pie(sizes, explode = explode, colors = colors, autopct = '%1.1f%%',shadow = False, startangle = 90)
        self.ax.legend(patches, labels, loc = "best")
        
    def update_pie(self, frame):  
        return self.ln
    
visP = PieVisualiser()
# Used a subscription to the /odom topic to have the possibility to spin the program automatically
subP = rospy.Subscriber('/clock', Clock, visP.pie_callback)
aniP = FuncAnimation(visP.fig1, visP.update_pie, init_func = visP.pie_init)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Path trajectory followed by the robot

In [13]:
class Visualiser:
    def __init__(self):
        self.fig, self.ax = plt.subplots() 
        self.ln, = self.ax.plot([], [], 'ro') 
        self.x_data, self.y_data = [] , []
        
    def plot_init(self): 
        self.ax.set_xlim(-10, 10) 
        self.ax.set_ylim(-10, 10) 
        
        return self.ln

    def odom_callback(self, msg): 
        self.y_data.append(msg.pose.pose.position.y) 
        self.x_data.append(msg.pose.pose.position.x)

    def update_plot(self, frame): 
        self.ln.set_data(self.x_data, self.y_data) 
        
        return self.ln

vis = Visualiser()
sub0 = rospy.Subscriber('/odom', Odometry, vis.odom_callback)
ani = FuncAnimation(vis.fig, vis.update_plot, init_func = vis.plot_init)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Laser scan in the robot frame

In [14]:
class LaserVisualiser:
    
    global angle_interval
    
    def __init__(self):
        self.figL, self.axL = plt.subplots(subplot_kw = {'projection': 'polar'})
        self.ln, = self.axL.plot([], [], 'r-') 
        self.intensity, self.theta = [], []
    
    def laser_init(self):
        self.axL.set_thetalim(-np.pi/2, np.pi/2)
        self.axL.set_rmax(16.5)
        self.axL.set_theta_zero_location("N")
        
        return self.ln
    
    def laser_callback(self, msg):        
        for i in plt.gca().lines + plt.gca().collections:
            i.remove()
            
        self.intensity = msg.ranges
        self.theta = np.arange(msg.angle_min, msg.angle_max + msg.angle_increment, msg.angle_increment)
        
        self.ln = self.axL.plot(self.theta, self.intensity, 'r-')
        
        return self.ln
    
    
    def update_laserPlot(self, frame):
        return self.ln
    
visL = LaserVisualiser()
subL = rospy.Subscriber('/scan', LaserScan, visL.laser_callback)
aniL = FuncAnimation(visL.figL, visL.update_laserPlot, init_func = visL.laser_init) 

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …